In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import stats
from scipy.sparse import dok_matrix
import networkit as nk
import time
import ast
from pymincut.pygraph import PyGraph

import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict

In [12]:
# Read the network
def readGraph(input_network, input_clustering, required_cluster_stats):
    # Read the clustering
    cluster_df = pd.read_csv(input_clustering, sep="\t", header=None, names=[
                             "node_id", "cluster_name"], dtype=str)
    
    # Read required cluster stats
    cluster_stats_df = pd.read_csv(required_cluster_stats)

    # Read the network
    elr = nk.graphio.EdgeListReader('\t', 0, continuous=False, directed=False)
    graph = elr.read(input_network)
    graph.removeMultiEdges()
    graph.removeSelfLoops()
    node_mapping_dict = elr.getNodeMap()
    # node_mapping_dict_reversed = {v: k for k, v in node_mapping_dict.items()}

    return graph, node_mapping_dict, cluster_df, cluster_stats_df


In [126]:
edge_list = '/Users/laharianne/Desktop/Research/lanne2_networks/plusO/SBM_OutlierCluster_samples/cit_patents/N_graph_edge_list.tsv'
clustering_file = '/Users/laharianne/Desktop/Research/lanne2_networks/Datasets/leidenClustering/cit_patents-leiden-empirical/leiden_res0.01_i2/S6_cit_patents_leiden.res0.01_i2_post_cm_filter.R.tsv'
required_cluster_stats = '/Users/laharianne/Desktop/Research/network_evaluation/imagine_something_like_this/cit_patents_input_stats/cluster_stats.csv'
graph, node_mapping, cluster_df,cluster_stats_df = readGraph(edge_list, clustering_file,required_cluster_stats)


In [127]:
print(graph.numberOfNodes())
print(graph.numberOfEdges())

3774768
15724960


In [129]:
clustering_dict = dict(
        zip(
            cluster_df['node_id'],
            cluster_df['cluster_name'],
        )
    )
#clustering_dict  = {node_id : cluster_id}

cluster_node_mapping = defaultdict(set)
for node, cluster in clustering_dict.items():
    cluster_node_mapping[cluster].add(node_mapping[node])

#cluster_node_mapping  = {cluster_id : set(node_iid)}



In [130]:
def remove_edges(G_c, edges_to_remove):
    for edge in edges_to_remove:
        G_c.removeEdge(edge[0], edge[1])

In [131]:
clustered_nodes = [node_mapping[v] for v in clustering_dict.keys()]
G_c = nk.graphtools.subgraphFromNodes(graph, clustered_nodes)

edges_to_remove = []
clustered_nodes_set = set(clustered_nodes)
# node_mapping = {node_id : node_iid}
# node_mapping_reversed = {node_iid : node_id}
node_mapping_reversed = {u:str(v) for v,u in node_mapping.items()}
for edge in G_c.iterEdges():
    if clustering_dict[node_mapping_reversed[edge[0]]] != clustering_dict[node_mapping_reversed[edge[1]]]:
        edges_to_remove.append(edge)

print(G_c.numberOfEdges())
remove_edges(G_c, edges_to_remove)
print(G_c.numberOfEdges())


11122511
6344577


In [132]:
# Ensuring minimum degree:
total_actual_cluster_edges = 0
cluster_count = 0
total_edges = 0
node_count = 0
for cluster_id in cluster_node_mapping.keys():
    # nodes = [node_mapping[v] for v in cluster_df[cluster_df['cluster_name']==cluster_id]['node_id']]
    nodes = list(cluster_node_mapping[cluster_id])
    # sub_graph = nk.graphtools.subgraphFromNodes(graph,nodes)
    # total_actual_cluster_edges += sub_graph.numberOfEdges()

    min_cut_required = int(cluster_stats_df[cluster_stats_df['cluster']==cluster_id]['connectivity'])
    
    
    for node in nodes:
        node_deg = G_c.degree(node)
        if node_deg<min_cut_required:
            node_count += 1
            deg_diff  = min_cut_required - node_deg
            total_edges += deg_diff
            selected = set()
            selected.add(node)
            while deg_diff>0:
                idx = np.random.choice(len(nodes), 1)
                edge_end = nodes[idx[0]]
                if edge_end not in selected:
                    G_c.addEdge(node, edge_end)
                    selected.add(edge_end)
                    deg_diff -= 1

    cluster_count += 1
    if cluster_count%1000 ==0:
        print(cluster_count)




1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000


In [133]:
print(total_edges)
print(total_actual_cluster_edges)
print(node_count)

292112
0
234475


In [135]:
print(graph.numberOfNodes())
print(graph.numberOfEdges())
print(G_c.numberOfNodes())
print(G_c.numberOfEdges())

3774768
15724960
2065327
6636689


In [136]:
cluster_ids = list(cluster_df['cluster_name'].unique())
# cluster_ids = ["0"]
for cluster_id in cluster_ids:
    # print(cluster_id)
    # nodes = [node_mapping[v] for v in cluster_df[cluster_df['cluster_name']==cluster_id]['node_id']]
    nodes = list(cluster_node_mapping[cluster_id])

    sub_graph = nk.graphtools.subgraphFromNodes(graph,nodes)
    cluster_nodes = list(sub_graph.iterNodes())
    # print("Cluster nodes = ", len(cluster_nodes))
    cluster_edges = list(sub_graph.iterEdges())
    min_cut_required = int(cluster_stats_df[cluster_stats_df['cluster']==cluster_id]['connectivity'])

    G = PyGraph(cluster_nodes, cluster_edges)
    mincut_result = G.mincut("cactus", "bqueue", True)

    partitions = mincut_result[:-1]
    cut_size = mincut_result[-1]
    count = 0
    cluster_nodes_copy = set(cluster_nodes.copy())
    while cut_size < min_cut_required and count<20:
        # if count>10:
        #     print("Cluster_id : " , cluster_id)
        #     # print("Current cut size: ", cut_size, "Number of cluster edges : ", len(cluster_edges))
        #     # sorted_partitions = sorted(partitions, key=lambda x: len(x), reverse=True)
        #     # larget_partition = sorted_partitions[0]
        #     # remaining_partitions = sorted_partitions[1:]
        #     # print([len(partition) for partition in partitions], remaining_partitions)
        #     break
        
        new_edges = []
        for i in range(len(partitions)):
            part_nodes = set.intersection(set(partitions[i]), cluster_nodes_copy)
            part_nodes_list = list(part_nodes)
            remaining_nodes = list(cluster_nodes_copy - part_nodes)
            if len(remaining_nodes)>0 and len(part_nodes)>0:
                node_idxs = np.random.choice(len(remaining_nodes), (min_cut_required-cut_size))
                for j in node_idxs:
                    part_node_idx = np.random.choice(len(part_nodes), 1)
                    n1 = int(part_nodes_list[part_node_idx[0]])
                    n2 = int(remaining_nodes[j])
                    # new_edges.append((n1,n2))
                    # cluster_edges.append((n1,n2))
                    graph.addEdge(n1,n2)
                    cluster_nodes_copy = cluster_nodes_copy - set([n1])
        # print(new_edges)

        # print("End of while loop!")
        # nodes = [node_mapping[v] for v in cluster_df[cluster_df['cluster_name']==cluster_id]['node_id']]
        sub_graph = nk.graphtools.subgraphFromNodes(graph,nodes)
        # cluster_nodes = list(sub_graph.iterNodes())

        # print("Cluster nodes = ", len(cluster_nodes), " Number of edges : ", sub_graph.numberOfEdges())
        cluster_edges = list(sub_graph.iterEdges())
        new_G = PyGraph(cluster_nodes, cluster_edges)
        new_mincut_result = new_G.mincut("cactus", "bqueue", True)
        partitions = new_mincut_result[:-1]
        cut_size = new_mincut_result[-1]

        # count += 1

    # nodes = [node_mapping[v] for v in cluster_df[cluster_df['cluster_name']==cluster_id]['node_id']]
    # nodes = list(cluster_node_mapping[cluster_id])

    # final_sub_graph = nk.graphtools.subgraphFromNodes(graph,nodes)
    # final_cluster_nodes = list(final_sub_graph.iterNodes())
    # # print("Cluster nodes = ", len(final_cluster_nodes))
    # final_cluster_edges = list(final_sub_graph.iterEdges())
    # final_G = PyGraph(final_cluster_nodes, final_cluster_edges)
    # final_mincut_result = final_G.mincut("cactus", "bqueue", True)

    # final_partitions = final_mincut_result[:-1]
    # final_cut_size = final_mincut_result[-1] 
    # print(cluster_id, " - Required Mincut! - ", min_cut_required, " - Final Mincut! - ", cut_size) 
    if(cut_size < min_cut_required):
            print("Exception : ", cluster_id, min_cut_required, cut_size)
            # print([len(partition) for partition in final_partitions])
            # print(final_partitions)
 
    

KeyboardInterrupt: 